In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

In [12]:

artworks = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')

In [13]:
# Select Columns.
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# Convert URL's to booleans.
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

# Drop films and some other tricky rows.
artworks = artworks[artworks['Department']!='Film']
artworks = artworks[artworks['Department']!='Media and Performance Art']
artworks = artworks[artworks['Department']!='Fluxus Collection']

# Drop missing data.
artworks = artworks.dropna()

In [14]:
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
artworks['YearAcquired'] = artworks.DateAcquired.dt.year

In [15]:
# Remove multiple nationalities, genders, and artists.
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = 'Multiple_Persons'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = 'Multiple_Nationalities'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

# Convert dates to start date, cutting down number of distinct examples.
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]

# Final column drops and NA drop.
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

# Create dummies separately.
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

# Concat with other variables, but artists slows this wayyyyy down so we'll keep it out for now
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

Y = artworks.Department

In [52]:
# Alright! We've done our prep, let's build the model.
# Neural networks are hugely computationally intensive.
# This may take several minutes to run.


# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(1000,))
mlp.fit(X, Y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [53]:
mlp.score(X, Y)

0.7060994634907347

In [ ]:
LPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [55]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, X, Y, cv=5)

array([0.5843915 , 0.62194426, 0.51334126, 0.44263699, 0.43659024])

In [16]:
#alpha
#Our network is 1000 neurons wide and one layer. (100, 4, ) would create a network with two layers, one 100 wide and the other 4.
#Lastly, we'll discuss the activation function. The activation function determines whether the output from an individual perceptron is binary or continuous. By default this is a 'relu', or 'rectified linear unit function' function. In the exercise we went through earlier we used this binary function, but we discussed the sigmoid as a reasonable alternative. The sigmoid (called 'logistic' by SKLearn because it's a 'logistic sigmoid function') allows for continuous variables between 0 and 1, which allows for a more nuanced model. It does come at the cost of increased computational complexit

mlp_1 = MLPClassifier(activation='logistic', alpha=0.001,
       hidden_layer_sizes=(100,))
mlp_1.fit(X, Y)

mlp_2 = MLPClassifier(activation='logistic', alpha=0.01,
       hidden_layer_sizes=(100,4,))
mlp_2.fit(X, Y)
mlp_2.score(X, Y)

mlp_3 = MLPClassifier(activation='logistic', alpha=0.1,
       hidden_layer_sizes=(500,))
mlp_3.fit(X, Y)
mlp_3.score(X, Y)

mlp_4 = MLPClassifier(activation='logistic', alpha=0.001,
       hidden_layer_sizes=(400,100,4,))
mlp_4.fit(X, Y)
mlp_4.score(X, Y)


0.6537232220996156

In [24]:
mlp_1.score(X, Y)

0.7051767436551121

In [25]:
mlp_2.score(X, Y)

0.660800578364598

In [26]:
mlp_3.score(X, Y)

0.6364388721890339

In [27]:
mlp_4.score(X, Y)

0.6537232220996156

In [20]:
cross_val_score(mlp_1, X, Y, cv=5)

array([0.60132211, 0.67468848, 0.49840666, 0.58490297, 0.50908572])

In [21]:
cross_val_score(mlp_2, X, Y, cv=5)

array([0.60346222, 0.66764958, 0.56161712, 0.54252283, 0.47769004])

In [22]:
cross_val_score(mlp_3, X, Y, cv=5)

array([0.57307272, 0.6462475 , 0.58040428, 0.55284437, 0.55936638])

In [23]:
cross_val_score(mlp_4, X, Y, cv=5)

array([0.6069815 , 0.65361933, 0.50344828, 0.58847032, 0.50822947])